In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pd_support import *
from scipy.spatial.transform import Rotation as R

from numba import njit
import itertools

from calculations_support import *

In [2]:
mc, mc_dt = read_df_csv("../data_csv/random_movement_sync_mc.csv")
cart_df = pd.read_csv("../data_csv/random_movement_sync.csv")

In [13]:
mc = add_datetime_col(mc, mc_dt, "seconds")
cart_df = add_datetime_diff(cart_df, mc_dt,"sync", "mils", True)
mc["time"][0]


,sys_time,e_fr,e_fl,e_rr,e_rl,rtc,mils,sync,time,av_fr,av_fl,av_rr,av_rl
607,2022-06-23 09:53:21.983740,-1,0,0,0,2019-01-01 05.31.27.000000 AM,88258,1,2022-06-23 09:53:21.904,0.0,0.0,0.0,0.0
608,2022-06-23 09:53:21.984242,-1,0,0,0,2019-01-01 05.31.27.000000 AM,88268,1,2022-06-23 09:53:21.914,0.0,0.0,0.0,0.0
609,2022-06-23 09:53:21.984764,-1,0,0,0,2019-01-01 05.31.27.000000 AM,88278,1,2022-06-23 09:53:21.924,0.0,0.0,0.0,0.0
610,2022-06-23 09:53:21.985264,-1,0,0,0,2019-01-01 05.31.27.000000 AM,88288,1,2022-06-23 09:53:21.934,0.0,0.0,0.0,0.0
611,2022-06-23 09:53:22.054762,-1,0,0,0,2019-01-01 05.31.27.000000 AM,88298,1,2022-06-23 09:53:21.944,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5408,2022-06-23 09:54:10.017709,-1454,251,-331,-1808,2019-01-01 05.32.15.000000 AM,136301,1,2022-06-23 09:54:09.947,0.0,0.0,0.0,0.0
5409,2022-06-23 09:54:10.018210,-1454,251,-331,-1808,2019-01-01 05.32.16.000000 AM,136311,1,2022-06-23 09:54:09.957,0.0,0.0,0.0,0.0
5410,2022-06-23 09:54:10.018712,-1454,251,-331,-1808,2019-01-01 05.32.16.000000 AM,136321,1,2022-06-23 09:54:09.967,0.0,0.0,0.0,0.0
5411,2022-06-23 09:54:10.018712,-1454,251,-331,-1808,2019-01-01 05.32.16.000000 AM,136331,1,2022-06-23 09:54:09.977,0.0,0.0,0.0,0.0


In [4]:
"""getting initial values of motion capture data"""

# marker_cen = np.array(mc[["center_x", "center_y", "center_z"]].iloc[0]).T
marker_xvec = np.array(mc[["xdir_x", "xdir_y", "xdir_z"]].iloc[0]).T
marker_zvec = np.array(mc[["zdir_x", "zdir_y", "zdir_z"]].iloc[0]).T
marker_org = np.array(mc[["org_x", "org_y", "org_z"]].iloc[0]).T
marker_org[0]

0.055481

In [5]:
def calculate_rotmat(xdir,zdir,org):
    """
    this function calculates rotation matrix
    """
    v1 = xdir - org #v1
    v2 = zdir - org #v2

    vxnorm = v1/np.linalg.norm(v1)

    vzcap = v2 - (vxnorm.T @ v2) * vxnorm
    vznorm = vzcap/ np.linalg.norm(vzcap)

    vynorm = np.cross(vznorm.T[0], vxnorm.T[0]).reshape(3,1)
    rotMat = np.hstack((vxnorm, vynorm, vznorm))
    return rotMat

In [6]:
v1 = marker_xvec - marker_org #v1
v2 = marker_zvec - marker_org #v2

v1 = v1.reshape(3,1)
v2 = v2.reshape(3,1)
v3 = marker_org.reshape(3,1)

In [7]:
rot_mat = calculate_rotmat(v1, v2, v3)

In [8]:
"""circumfrence of the wheel"""

cir_wheel = np.degrees(2*np.pi*(47.5/1000))
cir_wheel
radius = 47.5/1000

lx = 79 #half of the distance between the wheels
ly = 122.5/2

lx = lx/1000
ly = ly/1000

In [9]:
"""calculating angles"""
""" angular velocity"""
cart_df["av_fr"] = (cart_df["e_fr"]*0.09).diff()/0.01
cart_df["av_fl"] = (cart_df["e_fl"]*0.09).diff()/0.01
cart_df["av_rr"] = (cart_df["e_rr"]*0.09).diff()/0.01
cart_df["av_rl"] = (cart_df["e_rl"]*0.09).diff()/0.01
cart_df['av_fr'] = cart_df["av_fr"].fillna(0)
cart_df['av_fl'] = cart_df["av_fl"].fillna(0)
cart_df['av_rr'] = cart_df["av_rr"].fillna(0)
cart_df['av_rl'] = cart_df["av_rl"].fillna(0)

#converting them to radians
cart_df['av_fr'] = cart_df["av_fr"]* np.pi/180
cart_df['av_fl'] = cart_df["av_fl"]* np.pi/180
cart_df['av_rr'] = cart_df["av_rr"]* np.pi/180
cart_df['av_rl'] = cart_df["av_rl"]* np.pi/180

# cart_df["av_fl"]

In [10]:

_b1 = range(88, 91, 1)
_b2 = range(88, 91, 1)
_b3 = range(88, 91, 1)
_b4 = range(88, 91, 1)
_a1 = range(50, 53, 1)
_a2= range(50, 53, 1)
_a3= range(50, 53, 1)
_a4 = range(50, 53, 1)

pseudo_mat = calculate_combinations(_b1, _b2, _b3, _b4, _a1, _a2, _a3, _a4, radius)
pseudo_mat.shape

(6561, 3, 4)

In [16]:
def interpolate_data(target, reference, columns):

    """
    this function interpolates the dataframe target to the dataframe reference
    input is the dataframe target and reference
    both should have "time" column to them
    """

    _temp_df = pd.DataFrame(columns=columns)

    for i in columns:
        inp_fn = interp1d(target["time"], target[i], axis=0 ,fill_value='extrapolate')
        _temp_df[i] = inp_fn(reference["time"])

    _temp_df = _temp_df.fillna(0)
    return _temp_df

607    2022-06-23 09:53:21.904
608    2022-06-23 09:53:21.914
609    2022-06-23 09:53:21.924
610    2022-06-23 09:53:21.934
611    2022-06-23 09:53:21.944
                 ...          
5408   2022-06-23 09:54:09.947
5409   2022-06-23 09:54:09.957
5410   2022-06-23 09:54:09.967
5411   2022-06-23 09:54:09.977
5412   2022-06-23 09:54:09.987
Name: time, Length: 4806, dtype: datetime64[ns]

In [17]:
cart_inf = interpolate_data(cart_df, mc, ["av_fr", "av_fl", "av_rr", "av_rl"])

UFuncTypeError: ufunc 'true_divide' cannot use operands with types dtype('float64') and dtype('<m8[ns]')

In [ ]:
iterate_get_diff(pseudo_mat, cart_df, ["av_fr", "av_fl", "av_rr", "av_rl"], mc, ["org_z", "org_x"])

ValueError: Columns must be same length as key